In [ ]:
import smash
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def correct_bias(model: smash.Model, df: pd.DataFrame):
    model_correct = model.copy()

    for i, code in enumerate(model.mesh.code):
        if code in df["code"].values:
            df_code = df[df["code"]==code]
            model_correct.sim_response.q[i, df_code["timestep"].values] += df_code["bias"].values

    return model_correct

In [ ]:
model_train = smash.io.read_model("../models/ardeche-train.hdf5")
df_train = pd.read_csv("res/corrected-train.csv")
model_train_correct = correct_bias(model_train, df_train)

model_test = smash.io.read_model("../models/ardeche-test.hdf5")
df_test = pd.read_csv("res/corrected-test.csv")
model_test_correct = correct_bias(model_test, df_test)

In [ ]:
list_code = df_train["code"].unique()

In [ ]:
def score_dframe(model, model_correct, list_code):

    scores = {"code": [], "method": [], "nse": [], "kge": []}

    nse = smash.metrics(model, "nse")
    nse_correct = smash.metrics(model_correct, "nse")

    kge = smash.metrics(model, "kge")
    kge_correct = smash.metrics(model_correct, "kge")

    for i, c in enumerate(list_code):

        scores["code"].append(c) 
        scores["method"].append("classic")
        scores["nse"].append(nse[i])
        scores["kge"].append(kge[i])

        scores["code"].append(c) 
        scores["method"].append("lstm")
        scores["nse"].append(nse_correct[i])
        scores["kge"].append(kge_correct[i])

    return pd.DataFrame(scores)

In [ ]:
df1 = score_dframe(model_train, model_train_correct, list_code)
ax = df1.boxplot(column=["nse", "kge"], by="method")

In [ ]:
df2 = score_dframe(model_test, model_test_correct, list_code)
ax = df2.boxplot(column=["nse", "kge"], by="method")
plt.ylim([-1,1])

In [ ]:
df_test.boxplot(column="bias", by="code")